# NFT Wash Trading

Here we can build a lab notebook cell by cell.

The first task is to get us both on the same page on what data we'll be using for the project, and then we can work on loading it in and working with it.

## Data import

In [25]:
##Data import code here.
import os
import pandas as pd
cwd = os.getcwd()
bored_ape_location = cwd + '/data/bored_ape.csv'

In [26]:
#Load in the csv
bored_ape = pd.read_csv(bored_ape_location, low_memory=False)
bored_ape

,Unnamed: 0,approved_account,asset_id,asset_token_id,asset_num_sales,asset_background_color,asset_image_url,asset_image_preview_url,asset_image_thumbnail_url,asset_image_original_url,...,asset_bundle_assets_16_collection_telegram_url,asset_bundle_assets_16_collection_twitter_username,asset_bundle_assets_16_collection_instagram_username,asset_bundle_assets_16_collection_wiki_url,asset_bundle_assets_16_decimals,asset_bundle_assets_16_token_metadata,asset_bundle_assets_16_owner_user_username,asset_bundle_assets_16_owner_profile_img_url,asset_bundle_assets_16_owner_address,asset_bundle_assets_16_owner_config
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32309,307,NaN,23133633.0,2824.0,2.0,NaN,https://lh3.googleusercontent.com/kNc8S2n34Qd1...,https://lh3.googleusercontent.com/kNc8S2n34Qd1...,https://lh3.googleusercontent.com/kNc8S2n34Qd1...,https://ipfs.io/ipfs/QmQU6gQtV5s9h6TT3tJHhnFv2...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32310,308,NaN,23143625.0,9909.0,3.0,NaN,https://lh3.googleusercontent.com/4948N1jf9ih9...,https://lh3.googleusercontent.com/4948N1jf9ih9...,https://lh3.googleusercontent.com/4948N1jf9ih9...,ipfs://QmdDLMpmuL9a7yNspTVvbJi1bH21H6VPmxJkrzR...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32311,309,NaN,23136894.0,4936.0,4.0,NaN,https://lh3.googleusercontent.com/Q1qhmcwFm3JN...,https://lh3.googleusercontent.com/Q1qhmcwFm3JN...,https://lh3.googleusercontent.com/Q1qhmcwFm3JN...,https://ipfs.io/ipfs/QmXPfymDmpcwX2VN7gtZcCEKD...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32312,310,NaN,23115287.0,606.0,3.0,NaN,https://lh3.googleusercontent.com/LfEZ81VBu6tO...,https://lh3.googleusercontent.com/LfEZ81VBu6tO...,https://lh3.googleusercontent.com/LfEZ81VBu6tO...,https://ipfs.io/ipfs/QmZ7tTWNNy3sjmNvUkTigMJ6K...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#Infers some column data types to float and int to save some space
bored_ape_converted = bored_ape.infer_objects()

In [28]:
#Delete rows without wallet address
bored_ape_converted.dropna(subset=['winner_account_address'], inplace=True)

In [29]:
bored_ape_converted[['winner_account_address','winner_account_user_username','seller_user_username', 'seller_address']]

,winner_account_address,winner_account_user_username,seller_user_username,seller_address
126,0xfd8395c8d447b1813920c787d48eb73f970e8536,2LETTE_,AKC_Li,0x493a401122d858324517c5ec95145dc970c2c363
127,0x13928eb9a86c8278a45b6ff2935c7730b58ac675,BTCpirate,Zwarte_Kip,0x7e4225ed2c855d43b39f3b29d3790e9918d6b527
128,0x98e9de3364f73824b06fc6531c42b85e67a4a98c,01101010101010,gamblers_anonymous,0xd185e7e5e3a95e723736e2825ba0c2e9447f19d4
129,0xd065b1c5ad078d494f1bcd10e9c7fbf585c866ae,BAYCLove2,AllianceDAO,0x3a6bfe2b87b4b7d8ec76142d0ada23fce10b74b0
130,0x2e21f5d32841cf8c7da805185a041400bf15f21a,stani,JivaCore,0x33e3c007d1d48e2b645c9ce22570267b0c82f578
...,...,...,...,...
32309,0x329ad5b4e6e642b2d95af1b410e67ccb8f3c864f,Old9,jackmaschka,0x90f3146fa0e9e2056bbe383ecd2ff5864eaa78c5
32310,0xbd6c990981124c5027ad43fc8e6aa957f8800863,SeriousArtVault,4932114,0xd472b0798421159999e3db0aaa2d53bc0d7acfa3
32311,0x58791b7d2cfc8310f7d2032b99b3e9dffaae4f17,geovault,Benone_Vault,0x214a9b22e5435eeeaff7c06858969c8fdc220df5
32312,0x92458d6babe63c464fc5369d56b0dab864feabe8,LandVault,ajjaffy_vault,0x2c2ed4b3876c442fee80bee76ce0ee2ca2a512af


Usernames that are listed as NaN are unnamed on OpenSea but still have a wallet attached.

In [30]:
#Build a simple request on etherscan to scale up for each seller wallet in opensea data
import requests
#Arbitrary address for testing, when this is in a loop it should read from the panda/csv

address = '0x8323cc95c6fc88c832086e38869cfe1d834a4980'
payload = {'module':'account', 
           'action':'txlist', 
           'address':address,
           'startblock':'0',
           'endblock':'99999999',
           'page':'0',
           'offset':'0',
           'sort':'asc',
           'apikey':'5PN91CD3CXHJBPY4U3ZPCG86ZHWJUUZHBF'
          }
r = requests.get('https://api.etherscan.io/api', params=payload)
#Create a dict to store all the transactions
wallet_dict = {}
all_wallet_dict = {}

NameError: name 'address' is not defined

In [6]:
for transaction in r.json()['result']:
    if transaction['from'] == address:
        wallet_dict[transaction['to']] = 'sent'
    else:
        wallet_dict[transaction['from']] = 'recieved'
all_wallet_dict[address] = wallet_dict

In [7]:
#Shows the result, a dict with the key being the wallet address
#and the value being all transacations + type
print(all_wallet_dict)

{'0x8323cc95c6fc88c832086e38869cfe1d834a4980': {'0xc3fbc3f485f0d9b0bd21b13a4aaa8340160156cb': 'recieved', '0xa5409ec958c83c3f309868babaca7c86dcb077c1': 'sent', '0xba30e5f9bb24caa003e9f2f0497ad287fdf95623': 'sent', '0xedd3ee98fd6050d104ccf983cd13948557057a8f': 'sent', '0xd5f76a01d4ebe570a9252f1a8459b44fe8f4b5bd': 'sent', '0xfc007068c862e69213dc7aa817063b1803d4e941': 'sent', '0x45d8f7db9b437efbc74ba6a945a81aaf62dceda7': 'sent', '0x1b829b926a14634d36625e60165c0770c09d02b2': 'sent', '0xc2c747e0f7004f9e8817db2ca4997657a7746928': 'sent', '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b': 'sent', '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d': 'sent', '0x1b811fab3618e727d5d38d7e6338262ca372a3ca': 'sent', '0xfcc617a026bc7614db0debc00e72ea37d9ab9317': 'sent', '0x35a5add062f5d164bfecc8b5f2e5da10a09872e9': 'sent', '0xcde1efe3c576c900a257687aa95b2efad7150227': 'sent', '0x91bbc6dcfef7a30505106bf887528218cbda3f83': 'sent', '0xdbc410bd7a9a83c3cbe501a26cabada4a60826e2': 'sent', '0xb80a6253ebbdd2f72fe5b6fd0f8

In [8]:
all_wallet_list = []
for wallet_id in bored_ape_converted['winner_account_address']:
    all_wallet_list.append(wallet_id)
#Remove duplicates
all_wallet_list = set(all_wallet_list)
#all_wallet_list

# Dont know what the first nan value is but heres where I left off for the day.

## Analysis

I'll look at the paper again and determine a similar structure of analysis. Market by market, method by method.

In [9]:
##Analysis code here

## Results

In [10]:
## Hopefully some interesting results here